In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import matplotlib.pyplot as plt

In [2]:
anon_data = pd.read_csv("anonymized_shuffled_phase2.csv")
anon_data.reset_index(inplace=True) 
public_data = pd.read_excel("public_data_register_phase2.xlsx")
public_data.reset_index(inplace=True) 

In [3]:
public_data = public_data[public_data["last_voted"] != 2]
public_data["citizenship"] = np.where(public_data["citizenship"] != "Denmark", "Not danish", "Denmark")

In [4]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [5]:
def calculate_age_range(age):
    l_min = 5.28
    l_max = 7.03
    min_age = round(age + l_min)
    max_age = round(age - l_max)
    return [max_age, min_age]

In [6]:
age_lambda = lambda x: calculate_age(x["dob"])
age_range  = lambda x: calculate_age_range(x["age"])

In [7]:
public_data["age"] = public_data["dob"].apply(lambda x: calculate_age(x))
anon_data["age_range"] = anon_data["age"].apply(lambda x: calculate_age_range(x))

In [8]:
anon_data.head()

,index,sex,evote,age,zip,education,citizenship,party,age_range
0,0,Male,1,30.768506,2400,Vocational Education and Training (VET),Denmark,Green,"[24, 36]"
1,1,Female,0,47.177024,2400,Vocational Education and Training (VET),Denmark,Green,"[40, 52]"
2,2,Female,1,46.697183,2400,Vocational bachelors educations,Denmark,Red,"[40, 52]"
3,3,Female,0,41.181924,2300,Primary education,Denmark,Green,"[34, 46]"
4,4,Female,0,57.105226,2200,Vocational Education and Training (VET),Not danish,Green,"[50, 62]"


In [9]:
public_data.head()

,index,name,sex,dob,zip,citizenship,marital_status,last_voted,age
0,0,"al-Fawaz, Sabaah",Male,1996-08-23,2200,Denmark,Never married,1,24
1,1,"el-Asmar, Hasan",Male,1995-02-18,2100,Denmark,Never married,1,25
2,2,"Widaa, Cameron",Male,1996-08-24,2300,Denmark,Never married,1,24
3,3,"Santos, Jose",Male,2000-05-16,2300,Denmark,Never married,1,20
4,4,"Buczynski, Elizabeth",Female,1999-06-03,2300,Denmark,Never married,1,21


In [37]:
public_data[public_data["name"] == "Haley, Ashley"]

,index,name,sex,zip,citizenship,last_voted,age
898,898,"Haley, Ashley",Female,2200,Not danish,0,57


In [10]:
min(public_data["age"])

19

In [11]:
max(public_data["age"])

101

In [12]:
bins = [[19, 29], [30, 39], [40, 49], [50, 59], [60, 69], [70, 79], [80, 89], [90, 99], [100, 109]]

In [13]:
def ten_year_bins(age_range, bins):
    range = []
    diff = 0
    if type(age_range) == list:
        for idx, i in enumerate(bins):
            if age_range[0] <= i[1] and age_range[1] >= i[0]:
                diff_range = [max([age_range[0], i[0]]),  min([age_range[1], i[1]])]
                if (diff_range[1] - diff_range[0]) > diff:
                    range = bins[idx]
                    diff = diff_range[1] - diff_range[0]
    else:
        for idx, i in enumerate(bins):
            if age_range >= i[0] and age_range <= i[1]:
                return i
    return range

In [14]:
# anon_data["age_range_bins"] = anon_data["age_range"].apply(lambda x: ten_year_bins(x, bins))
# public_data["age_range_bins"] = public_data["age"].apply(lambda x: ten_year_bins(x, bins))

In [15]:
anon_data = anon_data.drop(columns=["age", "education"])
public_data = public_data.drop(columns=["marital_status", "dob"])

In [16]:
anon_data.head()

,index,sex,evote,zip,citizenship,party,age_range
0,0,Male,1,2400,Denmark,Green,"[24, 36]"
1,1,Female,0,2400,Denmark,Green,"[40, 52]"
2,2,Female,1,2400,Denmark,Red,"[40, 52]"
3,3,Female,0,2300,Denmark,Green,"[34, 46]"
4,4,Female,0,2200,Not danish,Green,"[50, 62]"


In [17]:
public_data.head()

,index,name,sex,zip,citizenship,last_voted,age
0,0,"al-Fawaz, Sabaah",Male,2200,Denmark,1,24
1,1,"el-Asmar, Hasan",Male,2100,Denmark,1,25
2,2,"Widaa, Cameron",Male,2300,Denmark,1,24
3,3,"Santos, Jose",Male,2300,Denmark,1,20
4,4,"Buczynski, Elizabeth",Female,2300,Denmark,1,21


In [18]:
# match_dict = dict()

# for index, row in public_data.iterrows():
#     match_dict[row["name"]] = []
#     public_str = row["sex"]+str(row["zip"])+row["citizenship"]+str(row["last_voted"])+str(row["age_range_bins"])
#     for index_anon, row_anon in anon_data.iterrows():
#         anon_str = row_anon["sex"]+str(row_anon["zip"])+row_anon["citizenship"]+str(row_anon["evote"])+str(row_anon["age_range_bins"])
#         if anon_str == public_str and row["age"] >= row_anon["age_range_bins"][0] and row["age"] <= row_anon["age_range_bins"][1]:
#             match_dict[row["name"]].append(row_anon["index"])

match_dict = dict()

for index, row in public_data.iterrows():
    match_dict[row["name"]] = []
    public_str = row["sex"]+str(row["zip"])+row["citizenship"]+str(row["last_voted"])
    for index_anon, row_anon in anon_data.iterrows():
        anon_str = row_anon["sex"]+str(row_anon["zip"])+row_anon["citizenship"]+str(row_anon["evote"])
        if anon_str == public_str and row["age"] >= (row_anon["age_range"][0]) and row["age"] <= (row_anon["age_range"][1]):
            match_dict[row["name"]].append(row_anon["index"])

In [19]:
dict_copy = match_dict.copy()

for key, value in dict_copy.items():
    if len(value) > 1 or len(value) == 0:
        del match_dict[key]

In [20]:
# match_dict

In [21]:
dict_copy_2 = match_dict.copy()
temp = dict()

for key, value in dict_copy_2.items():
    if value[0] not in temp:
        temp[value[0]] = 0
    else:
        temp[value[0]] += 1

for key, value in dict_copy_2.items():
    if temp[value[0]] != 0:
        del match_dict[key]

In [22]:
match_dict

{'Nubine, Tailor': [154],
 'el-Nasrallah, Saqr': [176],
 'al-Koroma, Huwaida': [98],
 'Fang, Christina': [131],
 'Lovan, Adinidiin': [117],
 'Paudel, Jijo': [174],
 'Lewis, Jordan': [160],
 'Strom, Hannah': [188],
 'al-Vaziri, Abbaas': [34],
 'Hamilton, Bryce': [110],
 'Bui, Samuel': [114],
 'Sayers, Aquasia': [198],
 'al-Rabbani, Ammaar': [28],
 'Wesley, Naaamorkor': [161],
 'Awtrey, Teryl': [64],
 'Thomas, Suzanna': [36],
 'Franko, Addison': [68],
 'Ricketts, Dwight': [169],
 'Le, Umeed': [29]}

In [23]:
users = []

for key, value in match_dict.items():
    users.append(value[0])

In [24]:
unique_users = anon_data[anon_data["index"].isin(users)]

In [25]:
unique_users

,index,sex,evote,zip,citizenship,party,age_range
28,28,Male,0,2300,Denmark,Green,"[14, 26]"
29,29,Male,0,2400,Denmark,Green,"[68, 80]"
34,34,Male,1,2400,Denmark,Green,"[59, 71]"
36,36,Female,0,2400,Not danish,Green,"[43, 55]"
64,64,Female,0,2200,Not danish,Red,"[27, 39]"
68,68,Male,0,2300,Denmark,Green,"[85, 97]"
98,98,Female,1,2400,Denmark,Green,"[30, 42]"
110,110,Male,1,2100,Not danish,Red,"[51, 63]"
114,114,Male,1,2400,Denmark,Green,"[62, 74]"
117,117,Female,1,2200,Not danish,Green,"[27, 39]"


In [26]:
user_dict = dict()

for index, row in unique_users.iterrows():
    u_user_string = row["sex"]+str(row["zip"])+row["citizenship"]+str(row["evote"])
    for index_2, value in public_data.iterrows():
        p_user_string = value["sex"]+str(value["zip"])+value["citizenship"]+str(value["last_voted"])
        if u_user_string == p_user_string and value["age"] >= (row["age_range"][0]) and value["age"] <= (row["age_range"][1]):
            if u_user_string in user_dict:
                user_dict[u_user_string] += 1
            else:
                user_dict[u_user_string] = 0


In [27]:
user_dict

{'Male2300Denmark0': 4,
 'Male2400Denmark0': 5,
 'Male2400Denmark1': 17,
 'Female2400Not danish0': 6,
 'Female2200Not danish0': 0,
 'Female2400Denmark1': 11,
 'Male2100Not danish1': 0,
 'Female2200Not danish1': 0,
 'Female2400Not danish1': 0,
 'Female2200Denmark1': 21,
 'Female2400Denmark0': 0,
 'Male2400Not danish0': 0,
 'Male2400Not danish1': 0,
 'Male2200Denmark1': 6,
 'Female2100Denmark1': 7}

In [28]:
user_dict_temp = []

for key, value in user_dict.items():
    if value == 0:
        user_dict_temp.append(key)

In [29]:
user_dict_temp

['Female2200Not danish0',
 'Male2100Not danish1',
 'Female2200Not danish1',
 'Female2400Not danish1',
 'Female2400Denmark0',
 'Male2400Not danish0',
 'Male2400Not danish1']

In [30]:
names_dict = dict()

for index, row in public_data.iterrows():
    u_user_string = row["sex"]+str(row["zip"])+row["citizenship"]+str(row["last_voted"])
    for key, value in anon_data.iterrows():
        u_user_string_2 = value["sex"]+str(value["zip"])+value["citizenship"]+str(value["evote"])
        for i in user_dict_temp:
            if u_user_string == u_user_string_2 and u_user_string_2 == i and row["age"] >= (value["age_range"][0]) and row["age"] <= (value["age_range"][1]):
                names_dict[row["name"]] = 0

In [31]:
names_dict

{'Fang, Christina': 0,
 'Lovan, Adinidiin': 0,
 'Paudel, Jijo': 0,
 'Plummer, Edgar': 0,
 'Chang, Kevin': 0,
 'Enriquez, Marcus': 0,
 'Hamilton, Bryce': 0,
 'Wesley, Naaamorkor': 0,
 'Wilson, Cheyenne': 0,
 'Taherkhani, Elisa': 0,
 'Szympruch, Sidnie': 0,
 'Barens, Hailee': 0,
 'Reinhardt, Abalean': 0,
 'Starr, Kassandra': 0,
 'Abruzzo, Amanda': 0,
 'Osborne, Michelle': 0,
 'Holmes, Michelle': 0,
 'al-Ben, Mahdeeya': 0,
 'Desantiago, Angelica': 0,
 'Kennedy, Sarah': 0,
 'Bullock, Amy': 0,
 'Holder, Shannon': 0,
 "Murphy, Bre'Anna": 0,
 'Fisher, Lorie': 0,
 'Tollerson, Bailee': 0,
 'Bost, Raelynn': 0,
 'Ariunbold, Alexa': 0,
 'Cerna, Gabreille': 0,
 'Sitton, Amanda': 0,
 'Awtrey, Teryl': 0,
 'Swanson, Quanishia': 0,
 'Anders, Indi': 0,
 'Smith, Jessica': 0,
 'Crapsey, Kristen': 0,
 'Castaneda, Minerva': 0,
 'Carson, Carly': 0,
 'El-Nasser, Jayln': 0,
 'Ross, Chloe': 0,
 'Livingston, Melanie': 0,
 'Folsom, Kendall': 0,
 'al-Hossain, Shameema': 0,
 'Farley, Asiala': 0,
 'Clanton, Quinita'

In [32]:
# dict_copy_2 = match_dict.copy()
# temp_list = dict()

# for key, value in dict_copy_2.items():
#     if value[0] in temp_list:
#         temp_list[value[0]].append(key)
#     else:
#         temp_list[value[0]] = [key]

In [33]:
# temp_copy = temp_list.copy()

# for key, value in temp_copy.items():
#     if len(value) > 1 or len(value) == 0:
#         del temp_list[key]

In [ ]:
# temp_list